## Analysis of the matrix extract usefull information

(glucose & octanoate)

In [8]:
import extFunc as ext
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import numpy as np
from importlib import import_module
import pandas as pd 
import cobra
import optlang
import copy 
import random
import scipy as sp

In [9]:
%%capture
#Load Models if not already in RAM

nameList = ['iJN1462_OCT_UR3_4.xml','iJN1462_GLC_UR6_3.xml']

if 'modelDict' not in locals():
    modelDict = ext.ImportFunction(nameList)

if 'glc_oct_comp'  not in locals():
    glc_oct_comp = ext.ModelComparison(modelDict=modelDict)


## Model preparation
-   loading Model, 
-   reseting upper lower bounds to max/min 1000
-   display all constrained Reactions --> warum atpm ?

In [10]:
#model_glu       = copy.deepcopy(modelDict['iJN1462_GLC_UR6_3']

model       = copy.deepcopy(modelDict['iJN1462_OCT_UR3_4'])


env_conditions = {r.id:(r.lower_bound, r.upper_bound)
 for r in model.reactions
 if (r.lower_bound not in [-1000, 0]) or (r.upper_bound not in [0, 1000])}
env_conditions


{'EX_ca2_e': (-10.0, 1000.0),
 'EX_cl_e': (-10.0, 1000.0),
 'EX_co2_e': (-100.0, 1000.0),
 'EX_cobalt2_e': (-10.0, 1000.0),
 'EX_cu2_e': (-10.0, 1000.0),
 'EX_fe2_e': (-10.0, 1000.0),
 'EX_h2o_e': (-100.0, 1000.0),
 'EX_h_e': (-100.0, 1000.0),
 'EX_hco3_e': (-10.0, 1000.0),
 'EX_k_e': (-10.0, 1000.0),
 'EX_mg2_e': (-10.0, 1000.0),
 'EX_mn2_e': (-10.0, 1000.0),
 'EX_mobd_e': (-10.0, 1000.0),
 'EX_na1_e': (-10.0, 1000.0),
 'EX_nh4_e': (-3.1, 1000.0),
 'EX_ni2_e': (-10.0, 1000.0),
 'EX_o2_e': (-13.5, 1000.0),
 'EX_octa_e': (-3.4, -3.4),
 'EX_pi_e': (-10.0, 1000.0),
 'EX_sel_e': (-10.0, 1000.0),
 'EX_so4_e': (-10.0, 1000.0),
 'EX_tungs_e': (-10.0, 1000.0),
 'EX_zn2_e': (-10.0, 1000.0),
 'SK_pqqA_kt_c': (-1.0, 0.0),
 'ATPM': (0.92, 0.92)}

## Matrix preparation
-   transfer Models to stöchiometric Matrices to perform linalg operations on it
-   Nu only uknowns are present, hence  O2, Nh4 & Octanoate Exchange are removed
-   Ns all reactions which are marked as boundary reactions are removed


In [15]:
N   = copy.deepcopy(model)
N   = cobra.util.create_stoichiometric_matrix(N,'dense')

Nu  = copy.deepcopy(model)
cobra.Model.remove_reactions(Nu,reactions=['EX_o2_e','EX_nh4_e','EX_octa_e'])
Nu  = cobra.util.create_stoichiometric_matrix( Nu ,'dense')

Ns   = copy.deepcopy(model)
Ns  = cobra.util.create_stoichiometric_matrix( Ns ,'dense')
L   = np.sum(N,axis=0)==1
Ns = np.delete(Ns, L,axis=1)

### Nullraum und Rang 

In [19]:
[nn,pp]     = N.shape
dim_Null    = pp-np.linalg.matrix_rank(N)
dim_lNull   = nn-np.linalg.matrix_rank(N)

print("dimmension Matrix =  " + str(N.shape))
print("dimmension rNull =  " + str(dim_Null))
print("dimmension lNull = " + str(dim_lNull))


dimmension Matrix =  (2153, 2927)
dimmension rNull =  841
dimmension lNull = 67


At least 841 reactions need to be measured in order to fully determine the system

In [ ]:
Kr = sp.linalg.null_space(N_u_glc)
Kl = sp.linalg.null_space(N_u_oct.T)


In [28]:
model.rep


[<Metabolite 10fthf[c] at 0x7fb5854f2c40>,
 <Metabolite 12dgr120[c] at 0x7fb5849ccf70>,
 <Metabolite 12dgr140[c] at 0x7fb5849ccf40>,
 <Metabolite 12dgr141[c] at 0x7fb5b5e00df0>,
 <Metabolite 12dgr160[c] at 0x7fb5b5e00bb0>,
 <Metabolite 12dgr160[e] at 0x7fb570124c70>,
 <Metabolite 12dgr161[c] at 0x7fb5b5e00c10>,
 <Metabolite 12dgr180[c] at 0x7fb5b5e00a30>,
 <Metabolite 12dgr180[e] at 0x7fb570124d30>,
 <Metabolite 12dgr181[c] at 0x7fb5b5e004f0>,
 <Metabolite 13dampp[c] at 0x7fb584e82f10>,
 <Metabolite 13dampp[e] at 0x7fb584e880a0>,
 <Metabolite 13dampp[p] at 0x7fb584e82e80>,
 <Metabolite 3hvacccoa[c] at 0x7fb585549c40>,
 <Metabolite 3ig3p[c] at 0x7fb584c5ca30>,
 <Metabolite 3mb2coa[c] at 0x7fb592ee5e80>,
 <Metabolite 3mb[c] at 0x7fb592f1f250>,
 <Metabolite 3mb[e] at 0x7fb57012a9d0>,
 <Metabolite 13dpg[c] at 0x7fb5b5c0ae50>,
 <Metabolite 15dap[c] at 0x7fb584e88160>,
 <Metabolite 2me4p[c] at 0x7fb5b5dd9220>,
 <Metabolite 2mecdp[c] at 0x7fb5b5df4ca0>,
 <Metabolite 3mb[p] at 0x7fb5927bae80>,

shortest path connecting a network buidling a bipartit network usind networkx --> shortest path algorithm